In [1]:
import gurobipy as gp
from gurobipy import GRB
import pickle
from copy import deepcopy
import random 
import numpy as np

In [2]:
# Nodes
n = 7
clients = [ i for i in range(n-1) if i !=0]
nodes = [0]+clients+[n-1]

In [ ]:
# not connected!
#arcs = [(0,1),(0,2), (0,5), (1,2),(1,3),(1,4),(1,5),(1,6),(2,3),(2,6),(3,4),(4,5),(5,6),(2,1),(3,1),(4,1),(5,1),(3,2),(4,3),(5,4)]
#cost = [11,10,10,5,15,15,5,10,15,10,20,15,10,5,15,15,5,15,20,15]
#c = { arcs[i]:  cost[i] for i in range(len(arcs))}

In [3]:
# fully connected
arcs = [(i,j) for i in nodes for j in nodes if i!=j]
cost = random.sample(range(20,200),len(arcs))
c = { arcs[i]:  cost[i] for i in range(len(arcs))}

In [4]:
np.random.seed(0)
q = {n:np.random.randint(10,15) for n in clients}
#q[0] = 0
#q[6] = 0
Q = 50



In [5]:
# Create Model
m = gp.Model('CVRP')

Restricted license - for non-production use only - expires 2023-10-25


In [6]:
# Adding variables
x = m.addVars(arcs,vtype = GRB.BINARY,name='x')         # x = arcs
u = m.addVars(clients,vtype = GRB.CONTINUOUS,name='u')  # u = clients


In [7]:
# Objective Function
m.setObjective(gp.quicksum(c[i,j]*x[i,j] for i,j in arcs),GRB.MINIMIZE)


In [8]:
#Constraints

# first constraint: check that each customer is visited once by a vehicle
m.addConstrs(gp.quicksum(x[i,j] for j in nodes if j!= i) == 1 for i in clients)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [9]:
# second constraint: check that if a vehicle visits a client, the vehicle leaves the client
m.addConstrs(gp.quicksum(x[i,j] for i in nodes if j!= i) == 1 for j in clients)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [10]:
# third constraint: avoid subtours (TO TAKE OUT LATER)
m.addConstrs((x[i,j]==1) >> (u[i]+q[j]== u[j]) for i,j in arcs if j!=0 and i!=0)

KeyError: 6

In [ ]:
print(len(arcs),clients)